In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

%cd ../..
from tools.mturk.mturk import MTurkHIT
from tools.mturk.spawn_experiment import get_verify_task_callback
from stimuli_generation import sg_utils as utils_stimuli_generation
%cd tools/data_analysis
from utils import utils_data
from utils import utils_analysis

In [ ]:
from matplotlib import rcParams

rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["DejaVu Sans"]

# output text as text and not paths
rcParams["svg.fonttype"] = "none"
rcParams["pdf.fonttype"] = "truetype"

colors = {
    "synthetic": [71 / 255, 120 / 255, 158 / 255], 
    "natural": [255 / 255, 172 / 255, 116 / 255],

    "natural easy": [255 / 255, 172 / 255, 116 / 255],
    "natural medium": [197 / 255, 135 / 255, 96 / 255],
    "natural hard": [150 / 255, 105 / 255, 75 / 255],
    "natural very hard": [109 / 255, 75 / 255, 52 / 255],

    "natural random": [255 / 255, 172 / 255, 116 / 255],
    "natural easiest": [255 / 255, 197 / 255, 140 / 255],
    "natural hardest": [197 / 255, 135 / 255, 96 / 255],
    #"natural hardest": [109 / 255, 75 / 255, 52 / 255],

    "synthetic easy": [71 / 255, 120 / 255, 158 / 255], 
    "synthetic medium": [52 / 255, 81 / 255, 105 / 255],
    "synthetic very hard": [39 / 255, 61 / 255, 79 / 255], 

    "natural low": [255 / 255, 172 / 255, 116 / 255],
    "natural high": [146 / 255, 100 / 255, 71 / 255],

    "synthetic low": [71 / 255, 120 / 255, 158 / 255], 
    "synthetic high": [39 / 255, 61 / 255, 79 / 255], 

    "c0": [245 / 255, 181 / 255, 121 / 255],
    "c1": [244 / 255, 170 / 255, 113 / 255],
    "c2": [203 / 255, 147 / 255, 94 / 255],
    "c3": [196 / 255, 134 / 255, 91 / 255],
    "c4": [150 / 255, 108 / 255, 68 / 255],
    "c5": [142 / 255, 98 / 255, 67 / 255],
    "c6": [116 / 255, 72 / 255, 44 / 255],
    "c7": [103 / 255, 76 / 255, 54 / 255],
    "c8":  [88 / 255, 73 / 255, 59 / 255],
}

# Load Data

In [ ]:
experiments = {
    "GoogLeNet (natural)": "data/experiment_202303/googlenet_natural_20230312",
    "GoogLeNet (synthetic)": "data/experiment_202303/googlenet_optimized_20230312",

    "DenseNet (natural)": "data/experiment_202303/densenet_201_natural_20230412",
    "DenseNet (synthetic)": "data/experiment_202303/densenet_201_optimized_20230503",

    "ResNet (natural)": "data/experiment_202303/resnet50_natural_20230325",
    "ResNet (synthetic)": "data/experiment_202303/resnet50_optimized_20230325",

    "Hard85 ResNet (natural)": "data/experiment_202303/resnet50_hard85_natural_20230505",
    "Hard85 ResNet (synthetic)": None,

    "Hard95 ResNet (natural)": "data/experiment_202303/resnet50_hard95_natural_20230510",
    "Hard95 ResNet (synthetic)": "data/experiment_202303/resnet50_hard95_optimized_20230514",

    "Hard99 ResNet (natural)": "data/experiment_202303/resnet50_hard99_natural_20230803",
    "Hard99 ResNet (synthetic)": None,

    "Robust ResNet (natural)": "data/experiment_202303/resnet50-l2_natural_20230314",
    "Robust ResNet (synthetic)": "data/experiment_202303/resnet50-l2_optimized_20230314",

    "Clip ResNet (natural)": "data/experiment_202303/clip-resnet50_natural_20230412",
    "Clip ResNet (synthetic)": "data/experiment_202303/clip-resnet50_optimized_20230502",

    "Hard85 Clip ResNet (natural)": "data/experiment_202303/clip-resnet50_hard85_natural_20230505",
    "Hard85 Clip ResNet (synthetic)": None,

    "Hard95 Clip ResNet (natural)": "data/experiment_202303/clip-resnet50_hard95_natural_20230510",
    "Hard95 Clip ResNet (synthetic)": None,

    "Hard99 Clip ResNet (natural)": "data/experiment_202303/clip-resnet50_hard99_natural_20230803",
    "Hard99 Clip ResNet (synthetic)": None,

    "WideResNet (natural)": "data/experiment_202303/wide_resnet50_natural_20230412",
    "WideResNet (synthetic)": "data/experiment_202303/wide_resnet50_optimized_20230502",

    "ViT (natural)": "data/experiment_202303/in1k-vit_b32_natural_20230501",
    "ViT (synthetic)": None,

    "Clip ViT (natural)": "data/experiment_202303/clip-vit_b32_natural_20230501",
    "Clip ViT (synthetic)": None,

    "ConvNeXT (natural)": "data/experiment_202303/convnext_b_natural_20230429",
    "ConvNeXT (synthetic)": "data/experiment_202303/convnext_b_optimized_20230509",

    "GoogLeNet DreamSim Easiest (natural)": "data/experiment_202303/googlenet_dreamsim_natural_easiest42_natural_20231111",
    "GoogLeNet DreamSim Easiest (synthetic)": None,

    "GoogLeNet DreamSim Hardest (natural)": "data/experiment_202303/googlenet_dreamsim_natural_hardest42_natural_20231111",
    "GoogLeNet DreamSim Hardest (synthetic)": None,

    "ResNet DreamSim Easiest (natural)": "data/experiment_202303/resnet50_dreamsim_natural_easiest42_natural_20231111",
    "ResNet DreamSim Easiest (synthetic)": None,

    "ResNet DreamSim Hardest (natural)": "data/experiment_202303/resnet50_dreamsim_natural_hardest42_natural_20231111",
    "ResNet DreamSim Hardest (synthetic)": None,
}

In [ ]:
dfs_results = {}
dfs_checks = {}

In [ ]:
for k in experiments:
    if (dfs_checks.get(k, None) is not None and dfs_results.get(k, None) is not None) and (len(dfs_checks.get(k, [])) > 0 and len(dfs_results.get(k, [])) > 0):
        continue

    if experiments[k] is None:
        dfs_results[k] = None
        dfs_checks[k] = None
        continue

    print(k)

    results = utils_data.load_results(experiments[k])
    structure = utils_data.load_and_parse_trial_structure(os.path.join(experiments[k], "structure.json"))
    df_results = utils_data.parse_results(results, use_raw_data=False)
    df_results = utils_data.append_trial_structure_to_results(df_results, structure)
    df_checks = utils_analysis.apply_all_checks(utils_data.parse_check_results(results))
    dfs_checks[k] = df_checks

    df_results = utils_data.append_checks_to_results(df_results, df_checks)
    dfs_results[k] = df_results

In [ ]:
reference_key = [k for k in experiments if k is not None][0]
reference_df_results = dfs_results[reference_key]
reference_df_checks = dfs_checks[reference_key]
for k in experiments:
    if experiments[k] is None:
        dfs_results[k] = reference_df_results.copy().head(0)
        dfs_checks[k] = reference_df_checks.copy().head(0)

### Load & apply list of flawed responses through multi participation

In [ ]:
import pickle
flawed_tasks_and_participants = pickle.load(open("data/experiment_202303_neurips_submission/flawed_tasks_and_participants.pd.pkl", "rb"))

In [ ]:
for k in dfs_results:
    dfs_results[k]["multi_participation"] = False

for _, row in flawed_tasks_and_participants.iterrows():
    task_id = row["task_id"]
    worker_id = row["worker_id"]

    # Find relevant responses
    for k in dfs_results:
        df_results = dfs_results[k]
        df_results.loc[(df_results["task_id"] == task_id) & (df_results["worker_id"] == worker_id), "multi_participation"] = True

In [ ]:
dfs_results_mp = {k: df[df["multi_participation"]] for k, df in dfs_results.items()}
dfs_results_no_mp = {k: df[~df["multi_participation"]] for k, df in dfs_results.items()}

dfs_results_main = {k: df[~df["catch_trial"] & ~df["is_demo"]] for k, df in dfs_results_no_mp.items()}
dfs_results_catch = {k: df[df["catch_trial"] & ~df["is_demo"]] for k, df in dfs_results_no_mp.items()}
dfs_results_demo = {k: df[df["is_demo"]] for k, df in dfs_results_no_mp.items()}

dfs_results_main_passed = {k: df[df["passed_checks"]] for k, df in dfs_results_main.items()}
dfs_results_catch_passed = {k: df[df["passed_checks"]] for k, df in dfs_results_catch.items()}
dfs_results_demo_passed = {k: df[df["passed_checks"]] for k, df in dfs_results_demo.items()}

dfs_results_main_rejected = {k: df[~df["passed_checks"]] for k, df in dfs_results_main.items()}
dfs_results_catch_rejected = {k: df[~df["passed_checks"]] for k, df in dfs_results_catch.items()}
dfs_results_demo_rejected = {k: df[~df["passed_checks"]] for k, df in dfs_results_demo.items()}

dfs_results_mp_main = {k: df[~df["catch_trial"] & ~df["is_demo"]] for k, df in dfs_results_mp.items()}
dfs_results_mp_catch = {k: df[df["catch_trial"] & ~df["is_demo"]] for k, df in dfs_results_mp.items()}
dfs_results_mp_demo = {k: df[df["is_demo"] == True] for k, df in dfs_results_mp.items()}

dfs_results_mp_main_passed = {k: df[df["passed_checks"]] for k, df in dfs_results_mp_main.items()}
dfs_results_mp_catch_passed = {k: df[df["passed_checks"]] for k, df in dfs_results_mp_catch.items()}
dfs_results_mp_demo_passed = {k: df[df["passed_checks"]] for k, df in dfs_results_mp_demo.items()}

dfs_results_mp_main_rejected = {k: df[~df["passed_checks"]] for k, df in dfs_results_mp_main.items()}
dfs_results_mp_catch_rejected = {k: df[~df["passed_checks"]] for k, df in dfs_results_mp_catch.items()}
dfs_results_mp_demo_rejected = {k: df[~df["passed_checks"]] for k, df in dfs_results_mp_demo.items()}

dfs_results_mp_or_rejected = {k: df[
    ~df["passed_checks"] | df["multi_participation"]] for k, df in dfs_results.items()}
dfs_results_main_mp_or_rejected = {k: df[~df["catch_trial"] & ~df["is_demo"] & (
    ~df["passed_checks"] | df["multi_participation"])] for k, df in dfs_results.items()}
dfs_results_catch_mp_or_rejected = {k: df[df["catch_trial"] & (
    ~df["passed_checks"] | df["multi_participation"])] for k, df in dfs_results.items()}
dfs_results_demo_mp_or_rejected = {k: df[df["is_demo"] & (
    ~df["passed_checks"] | df["multi_participation"])] for k, df in dfs_results.items()}

In [ ]:
print("Number of all responses:", sum([len(df) for df in dfs_results.values()]))
print("Number of all clean responses:", sum([len(df) for df in dfs_results_no_mp.values()]))
print("Number of all flawed responses:", sum([len(df) for df in dfs_results_mp.values()]))
print()

print("Number of all clean responses:", sum([len(df) for df in dfs_results_no_mp.values()]))
print("Number of all clean main responses:", sum([len(df) for df in dfs_results_main.values()]))
print("Number of all clean passed main responses:", sum([len(df) for df in dfs_results_main_passed.values()]))
print()

print("Number of all flawed responses:", sum([len(df) for df in dfs_results_mp.values()]))
print("Number of all flawed main responses:", sum([len(df) for df in dfs_results_mp_main.values()]))
print("Number of all flawed passed main responses:", sum([len(df) for df in dfs_results_mp_main_passed.values()]))
print()

print("Number of all flawed/rejected responses:", sum([len(df) for df in dfs_results_mp_or_rejected.values()]))
print("Number of all flawed/rejected main responses:", sum([len(df) for df in dfs_results_main_mp_or_rejected.values()]))
print()

print("Number of all participants:", sum([len(df["worker_id"].unique()) for df in dfs_results.values()]))
print("Number of clean participants:", sum([len(df["worker_id"].unique()) for df in dfs_results_no_mp.values()]))
print("Number of flawed/rejected participants:", sum([len(df["worker_id"].unique()) for df in dfs_results_mp_or_rejected.values()]))

## Analyze how many responses have been collected/accepted per unit/model

In [ ]:
dfs = dfs_results_main_passed
meta_datas = []
for k in dfs:
    if len(dfs[k]) == 0:
        continue
    n_units = dfs[k].groupby(["layer", "channel"]).mean(numeric_only=True).shape[0]
    n_responses_counts = dfs[k].groupby(["layer", "channel"])["correct"].apply(lambda df: len(df)).value_counts().to_dict()
    meta_datas.append({
        "model_condition": k,
        "units": n_units,
        "responses_per_unit": sorted(list(n_responses_counts.keys())),
        "completed": (n_responses_counts.get(31, 0) == n_responses_counts.get(32, 0) == 40 if n_units == 80 else
            n_responses_counts.get(30, 0) == 84 if n_units == 84 else False)
        })
meta_datas = pd.DataFrame(meta_datas)
meta_datas = meta_datas.set_index("model_condition")

print("All Experiments")
display(meta_datas)

print("Incomplete Experiments")
display(meta_datas[meta_datas["completed"] == False])

# Visualizations

In [ ]:
# Set this to True to use data of participants that participated in multiple experiments and
# data of participants that did not pass the checks.
use_flawed_data = False

if use_flawed_data:
    results_dir = "results_rejected"
else:
    results_dir = "results"

os.makedirs(results_dir, exist_ok=True)

### Calculate Confidence Intervals/Stds/SEMs

In [ ]:
import collections

if use_flawed_data:
    dfs = {k: dfs_results_main_mp_or_rejected[k].copy() for k in dfs_results_main_passed}
else:
    dfs = {k: dfs_results_main_passed[k].copy() for k in dfs_results_main_passed}


dummy_test_results = collections.namedtuple("DummyTestResults", ("confidence_interval",))((np.nan, np.nan))

import scipy
confidences = {k: scipy.stats.bootstrap(
    dfs[k].groupby(["layer", "channel"]).mean(numeric_only=True)["correct"].values.reshape(  # .groupby("participant_id").mean(numeric_only=True)
    (1, -1)), statistic=np.mean, n_resamples=1_0_000) if experiments[k] is not None else dummy_test_results for k in dfs}

means = {k: dfs[k].groupby(["layer", "channel"]).mean(numeric_only=True).mean(numeric_only=True) for k in experiments}
stds = {k: dfs[k].groupby(["layer", "channel"]).mean(numeric_only=True).std(numeric_only=True) for k in experiments}
sems = {k: stds[k] / np.sqrt(len(dfs[k])) if len(dfs[k]) > 0 else stds[k] for k in experiments}

In [ ]:
def plot_performance(means, confidences, relevant_experiments, labels=("Natural", "Synthetic"), color_names=("natural", "synthetic"), legend: bool = True,
                     chance_level: bool = True, xticks: bool = True, yticks: bool = True, xlabel="Model", ylabel="Proportion Correct",
                     bar_width: float = 1, grid: bool = True, rotate_ticks: bool = False, y_max: float = 0.9, legend_cols: int = 2,
                     show_errorbars: bool = True, ax=None):
    x = np.arange(len(relevant_experiments) // len(labels) * (len(labels) + 1), dtype=float)

    relevant_means = [means[k]["correct"] for k in relevant_experiments]
    error_values = np.array([
        (max(0, -confidences[k].confidence_interval[0] + means[k]["correct"]),
        max(0, confidences[k].confidence_interval[1] - means[k]["correct"]))
        for k in relevant_experiments]).T
    #error_values = np.array([2 * sems[k]["correct"] for k in experiments])
    #error_values = np.array([stds[k]["correct"] for k in experiments])
    missing_indices = [i for i, it in enumerate(relevant_means) if np.isnan(it)]
    #relevant_means = [it for i, it in enumerate(relevant_means) if i not in missing_indices]
    #error_values = error_values[:, [i for i in range(len(relevant_means)) if i not in missing_indices]]
    for i in missing_indices:
        group_idx = i // len(labels)
        element_idx = i % len(labels)
        j = group_idx * (len(labels) + 1) + element_idx
        x[j + 1:] = x[j:-1]

    
    for i in range(1, len(relevant_experiments) // len(labels)):
        x[i * (len(labels) + 1):] -= 0.5

    x = np.vstack([x[i::len(labels) + 1] for i in range(len(labels))]).reshape((-1,), order='F')

    x *= bar_width

    if ax is None:
        _, ax = plt.subplots(1, 1, figsize=(1 + bar_width*0.6*(len(relevant_means) - len(missing_indices)), 4.8))
    else:
        print("Reusing existing axis. Ensure that the axis has the right size:", (1 + bar_width*0.6*(len(relevant_means) - len(missing_indices)), 4.8))
    ax.bar(
        x,
        [means[k]["correct"] for k in relevant_experiments],
        color=[colors[color_names[i % len(labels)].lower()] for i in range(len(relevant_experiments))],
        width=bar_width
        )
    if show_errorbars:
        ax.errorbar(x, relevant_means, 
                    error_values, fmt=".", color="k")

    if xticks:
        single_bar_offset = bar_width * np.array([0.5 if np.isnan(means[k]["correct"]) else 0 for k in relevant_experiments]).reshape(-1, len(labels)).sum(-1)
        x_ticks = (x[:-1:len(labels)] + x[len(labels) - 1::len(labels)]) / 2 - single_bar_offset
        ax.set_xticks(x_ticks, [k.split("(")[0].replace(" ", "\n") for k in relevant_experiments][::len(labels)], rotation=45 if rotate_ticks else 0)
    else:
        ax.set_xticks([])

    if not yticks:
        ax.set_yticks([])

    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel, labelpad=-10)
    ax.set_ylim((0.475, y_max))
    
    x_max = max([x[i] for i, k in enumerate(relevant_experiments) if not np.isnan(means[k]["correct"])])

    if legend:
        hdls = [ax.scatter([], [], color=colors[k.lower()]) for k in color_names]
        ax.legend(hdls, labels, frameon=False, ncol=legend_cols)

    if chance_level:
        ax.text(x[1] + 0.6 if len(x) > 2 else x[0] - bar_width, 0.505, "Chance")
        ax.hlines(0.5, x.min() - 1, x.max() + 1, color="k", ls="-.", lw=1)

    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.set_xlim(-1.0 * bar_width, x_max + 0.75 * bar_width)
    ax.spines["bottom"].set_bounds((-0.75 * bar_width, x_max + 0.75 * bar_width))

    if grid:
        ax.set_axisbelow(True)
        ax.grid(linestyle="dashed", axis="y")

    plt.tight_layout()

# Get Hardest/Easiest Units

In [ ]:
# Get the best/worst units per model
joined_df = pd.DataFrame(columns=("model", "layer", "channel", "correct"))
for model in dfs_results:
    df = dfs_results[model]
    units_df = df[["layer", "channel", "correct"]].groupby(["layer", "channel"]).mean(numeric_only=True).reset_index(drop=False).sort_values("correct", ascending=False).copy()
    units_df["model"] = model
    joined_df = pd.concat((joined_df, units_df.iloc[:3], units_df.iloc[-3:]), axis=0, ignore_index=True)
joined_df = joined_df.set_index(["model", "layer", "channel"])

# DreamSim Intervention

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3.88, 4.8))

relevant_experiments = [
    "GoogLeNet DreamSim Hardest (natural)",
    "GoogLeNet (natural)",
    "GoogLeNet DreamSim Easiest (natural)",

    "ResNet50 DreamSim Hardest (natural)",
    "ResNet (natural)",
    "ResNet50 DreamSim Easiest (natural)",
]
plot_performance(means, confidences, relevant_experiments, ("Hardest", "Random", "Easiest"),
                 color_names=("natural hardest", "natural random", "natural easiest"),  bar_width=0.6,
                ax=ax, legend_cols=1)
ax.set_ylim(ax.get_ylim()[0], 1.0)
ax.set_xticklabels(("GoogLeNet", "ResNet"))

# Uncomment to get plots separately.
plt.tight_layout()
plt.savefig("results_metrics/unit_comparison_easiest_hardest_dreamsim_logistic_regression_sklearn.pdf", bbox_inches="tight")
plt.show()